In [27]:
import requests
import time

def search_reddit(query, limit=100, sort='relevance', timeframe='all'):
    query = query.replace(' ', '+')  # URL-encode spaces
    base_url = f"https://www.reddit.com/search.json?q={query}&limit={limit}&sort={sort}&t={timeframe}"
    
    try:
        response = requests.get(base_url, headers={'User-agent': 'yourbot'})
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None
    
    return response.json()

def get_post_comments(post_url):
    headers = {'User-agent': 'yourbot'}
    try:
        response = requests.get(post_url + ".json", headers=headers)
        response.raise_for_status()
        data = response.json()
        
        comments = []
        for comment in data[1]['data']['children']:
            if 'body' in comment['data']:
                comments.append(comment['data']['body'])
        
        return comments
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching comments: {e}")
        return []


def is_reviewer_style_comment(text, min_word_count=50):
    """
    Filters whether a comment looks like a review based on content and length.
    """
    review_keywords = ['i tried', 'i tested', 'in my experience', 'this product', 
                       'works great', 'did not like', 'pros and cons', 'overall', 'would recommend', 'not worth']
    
    text_lower = text.lower()
    if any(keyword in text_lower for keyword in review_keywords) or len(text.split()) >= min_word_count:
        return True
    return False


def get_all_comments(posts_json, post_limit=10):
    posts_with_comments = []
    
    for i, post in enumerate(posts_json['data']['children']):
        if i >= post_limit:
            break
        
        post_url = "https://www.reddit.com" + post['data']['permalink']
        title = post['data']['title']
        
        print(f"Fetching comments for: {title}")
        comments = get_post_comments(post_url)

        filtered_comments = []  # Store only reviewer-style comments

        for comment_text in comments:
            if is_reviewer_style_comment(comment_text):
                filtered_comments.append(comment_text)

        if filtered_comments:  # Only save posts that have at least one good comment
            posts_with_comments.append({
                "title": title,
                "url": post_url,
                "comments": filtered_comments
            })
        
        time.sleep(1)  # small pause to avoid getting blocked
    
    return posts_with_comments


# --- Main flow ---
query = input("Enter your search: ")

# Search Reddit
search_results = search_reddit(query, limit=20, sort='top', timeframe='month')

# Fetch comments from top posts
all_posts_with_comments = get_all_comments(search_results, post_limit=5)

# Show results
for post in all_posts_with_comments:
    print("\nTITLE:", post['title'])
    for comment in post['comments']:
        print("-", comment)


Fetching comments for: My trip to Antarctica: icebergs, penguins, sleeping on the ice etc.
Fetching comments for: Underwater footage of an Earthquake captured by scuba divers
Fetching comments for: Future Netherite Side-Grade Ideas
Fetching comments for: He could have imagined any other character in that scuba diving gear but he chose the Lioness
Fetching comments for: It’s a hygiene thing

TITLE: My trip to Antarctica: icebergs, penguins, sleeping on the ice etc.
- About you sleeping on the ice - how cold was it? And, what clothes have you worn (including shoes) to protect you from freezing? Seems like quite low temps below freezing but you all look like it is a casual hike. Lol. The yellow jackets - is it all from the ship store you all got or you bought it ahead of time? 

Great pictures and story! Thanks for sharing!
- One of the most amazing trips ever. We were literally eye to eye with curious humpbacks who came by the zodiac to “play” with us. We could have touched them they wer

In [18]:
all_posts_with_comments

[]

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("indexed_comments.csv")
df

,title,comment,_id
0,Looking for high waisted hiking pants for hot ...,I see you're getting a lot of recs to buy men'...,rec0
1,Most breathable and quick-drying women's hikin...,Not sure what's wrong with the linen pants if ...,rec1
2,Extremely hot weather: Merino or synthetics?,I did a multi-day route through the Grand Cany...,rec2
3,Extremely hot weather: Merino or synthetics?,I personally like both. A lightweight merino s...,rec3
4,Which Fjallraven Pants are the Best for Hiking...,"I am also going on a hike this summer, so I or...",rec4
5,Which pants for summer travel and hot weather?,First I would recommend the [**Renegade**](htt...,rec5
6,Dressing for hot weather in conservative Islam...,"My time to shine! I live in Dubai, frequently ...",rec6
7,sewing for hot weather travel,For making your own gear I recommend the subre...,rec7
8,sewing for hot weather travel,If you're going somewhere hot and humid I woul...,rec8
9,Light alpine hiking pants for summer use?,I don't know if you're EU or USA based but I u...,rec9
